In [6]:
# Import libraries and dataset

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

spam_data = pd.read_csv('./spam.csv',encoding = "latin-1", engine='python')

spam_data = spam_data.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1)
# spam_data.head(5)
spam_data.describe()
# spam_data.groupby('Class').describe()

,Class,Text
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [2]:
cv=CountVectorizer()
word_count_vector=cv.fit_transform(spam_data.Text)
word_count_vector.shape


(5572, 8672)

In [3]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer=TfidfTransformer()
tfidf_transformer.fit(word_count_vector)

df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names(),columns=["idf_weights"])

df_idf.sort_values(by=['idf_weights'])


,idf_weights
to,2.198545
you,2.254829
the,2.689346
in,2.933605
and,2.947347
...,...
bleh,8.932542
mee,8.932542
blimey,8.932542
mirror,8.932542


In [4]:
count_vector=cv.transform(spam_data.Text)
tf_idf_vector=tfidf_transformer.transform(count_vector)

feature_names = cv.get_feature_names()
h = 0
for x in tf_idf_vector:
    h+= 1
    df = pd.DataFrame(x.T.todense(), index=feature_names, columns=["tfidf"])
    print(df.sort_values(by=["tfidf"],ascending=False))
    if h == 3:
        break



                tfidf
jurong       0.326425
amore        0.326425
buffet       0.311608
bugis        0.275765
cine         0.275765
...               ...
electricity  0.000000
elections    0.000000
election     0.000000
eldest       0.000000
ûówell       0.000000

[8672 rows x 1 columns]
             tfidf
oni       0.546588
joking    0.523646
wif       0.431601
lar       0.408299
ok        0.272120
...            ...
election  0.000000
eldest    0.000000
elaya     0.000000
elama     0.000000
ûówell    0.000000

[8672 rows x 1 columns]
                      tfidf
fa                 0.460253
entry              0.352710
08452810075over18  0.230126
2005               0.222362
21st               0.222362
...                     ...
electricity        0.000000
elections          0.000000
election           0.000000
eldest             0.000000
ûówell             0.000000

[8672 rows x 1 columns]


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.pipeline import Pipeline


# messages = pd.read_csv('./spam.csv', encoding='latin-1')
# messages.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1,inplace=True)
# messages = messages.rename(columns={'v1': 'Class','v2': 'Text'})


x_train, x_test, y_train, y_test = train_test_split(spam_data['Text'],spam_data['Class'],test_size=0.2)

model = Pipeline([
    ('bow',CountVectorizer()),
    ('tfidf',TfidfTransformer()),
    ('classifier',MultinomialNB())
])

model.fit(x_train,y_train)

prediction = model.predict(x_test)

print(classification_report(y_test,prediction))

              precision    recall  f1-score   support

         ham       0.94      1.00      0.97       939
        spam       1.00      0.66      0.80       176

    accuracy                           0.95      1115
   macro avg       0.97      0.83      0.88      1115
weighted avg       0.95      0.95      0.94      1115

